In [1]:
"""
parameter_test_stardist.py

Standalone script to test multiple StarDist parameters on a single H&E image and save overlay results.
"""
import os
from pathlib import Path
import tifffile
from csbdeep.utils import normalize
from stardist.models import StarDist2D
import matplotlib.pyplot as plt

2025-05-19 14:26:27.819501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747689987.834520   13841 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747689987.839351   13841 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747689987.851489   13841 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747689987.851542   13841 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747689987.851544   13841 computation_placer.cc:177] computation placer alr

In [2]:
# -----------------------------
# Configuration
# -----------------------------
# Path to the F07839 H&E TIFF image
IMAGE_PATH = Path(
    "/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/images_for_alignments"
    "/121724-121924_RL_mRT_TMA4_1_TMA5_1/hne/121724_RL_mRT_TMA4_Slide_1.tif"
)
# Directory to save parameter test overlays
OUTPUT_DIR = Path(
    "/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test"
)

In [10]:
# Normalization percentiles
MIN_PERCENTILE = 5
MAX_PERCENTILE = 95

# StarDist big-segmentation tile settings
BLOCK_SIZE = 2048      # tile size (px)
MIN_OVERLAP = 256      # overlap between tiles
CONTEXT = 94           # extra border context

# Parameter grids to test
PROB_THRESHOLDS = [0.33, 0.25, 0.20, 0.15]
NMS_THRESHOLDS = [0.10, 0.05, 0.03]

# -----------------------------
# Ensure output directory exists
# -----------------------------
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [11]:
# -----------------------------
# Load and normalize image
# -----------------------------
print(f"Loading image: {IMAGE_PATH}")
img = tifffile.imread(str(IMAGE_PATH))
img_norm = normalize(img, MIN_PERCENTILE, MAX_PERCENTILE, axis=(0,1,2))

# -----------------------------
# Load pre-trained StarDist model
# -----------------------------
print("Loading StarDist model: 2D_versatile_he")
model = StarDist2D.from_pretrained("2D_versatile_he")

Loading image: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/images_for_alignments/121724-121924_RL_mRT_TMA4_1_TMA5_1/hne/121724_RL_mRT_TMA4_Slide_1.tif
Loading StarDist model: 2D_versatile_he
Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [12]:
# -----------------------------
# Parameter sweep
# -----------------------------
for p in PROB_THRESHOLDS:
    for nms in NMS_THRESHOLDS:
        print(f"Running segmentation with prob_thresh={p}, nms_thresh={nms}")
        labels, polys = model.predict_instances_big(
            img_norm,
            axes='YXC',
            block_size=BLOCK_SIZE,
            min_overlap=MIN_OVERLAP,
            context=CONTEXT,
            prob_thresh=p,
            nms_thresh=nms
        )

        # Overlay and save
        fig, ax = plt.subplots(figsize=(10,10))
        ax.imshow(img)
        ax.imshow(labels, cmap='jet', alpha=0.3)
        ax.axis('off')

        out_file = OUTPUT_DIR / f"F07839_p{p}_nms{nms}_overlay.png"
        fig.savefig(str(out_file), dpi=900, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        print(f"Saved overlay: {out_file}")

print("Parameter test complete.")

Running segmentation with prob_thresh=0.33, nms_thresh=0.1
effective: block_size=(2048, 2048, 3), min_overlap=(256, 256, 0), context=(96, 96, 0)


100%|███████████████████████████████████████████████████████████████████████████████| 56/56 [00:55<00:00,  1.00it/s]


Saved overlay: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test/F07839_p0.33_nms0.1_overlay.png
Running segmentation with prob_thresh=0.33, nms_thresh=0.05
effective: block_size=(2048, 2048, 3), min_overlap=(256, 256, 0), context=(96, 96, 0)


100%|███████████████████████████████████████████████████████████████████████████████| 56/56 [00:57<00:00,  1.02s/it]


Saved overlay: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test/F07839_p0.33_nms0.05_overlay.png
Running segmentation with prob_thresh=0.33, nms_thresh=0.03
effective: block_size=(2048, 2048, 3), min_overlap=(256, 256, 0), context=(96, 96, 0)


...

Saved overlay: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test/F07839_p0.15_nms0.1_overlay.png
Running segmentation with prob_thresh=0.15, nms_thresh=0.05
effective: block_size=(2048, 2048, 3), min_overlap=(256, 256, 0), context=(96, 96, 0)


100%|███████████████████████████████████████████████████████████████████████████████| 56/56 [01:10<00:00,  1.26s/it]


Saved overlay: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test/F07839_p0.15_nms0.05_overlay.png
Running segmentation with prob_thresh=0.15, nms_thresh=0.03
effective: block_size=(2048, 2048, 3), min_overlap=(256, 256, 0), context=(96, 96, 0)


100%|███████████████████████████████████████████████████████████████████████████████| 56/56 [01:17<00:00,  1.38s/it]


Saved overlay: /mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/CAR_T/data/cell_segmentation/parameter_test/F07839_p0.15_nms0.03_overlay.png
Parameter test complete.
